# Chains in LangChain

## Outline

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

In [3]:
!pip install python-dotenv
import warnings
warnings.filterwarnings('ignore')


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [4]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

In [5]:
#!pip install pandas

In [6]:
import pandas as pd
df = pd.read_csv('Data.csv')

In [7]:
df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\n,I loved this product. But they only seem to l...


## LLMChain

In [8]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain


from os import listdir
from os.path import isfile, join
from langchain.chains import LLMChain
from langchain.llms import HuggingFaceTextGenInference
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate

In [9]:
#llm = ChatOpenAI(temperature=0.9)


In [10]:
# LLM Inference Server URL
inference_server_url = "http://hf-mistral-7b.llm-model-collection.svc.cluster.local:3000/"

# LLM definition
llm = HuggingFaceTextGenInference(
    inference_server_url=inference_server_url,
    max_new_tokens=512,
    top_k=10,
    top_p=0.95,
    typical_p=0.95,
    temperature=0.01,
    repetition_penalty=1.03,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)

In [11]:
prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

In [12]:

chain = LLMChain(llm=llm, prompt=prompt)

In [13]:
product = "Queen Size Sheet Set"
chain.run(product)



AI: A company that makes Queen Size Sheet Sets could be called "Queen Size Sheets Co." or "Queen Size Bedding Co."

'\n\nAI: A company that makes Queen Size Sheet Sets could be called "Queen Size Sheets Co." or "Queen Size Bedding Co."'

## SimpleSequentialChain

In [14]:
from langchain.chains import SimpleSequentialChain

In [15]:
#llm = ChatOpenAI(temperature=0.9)

# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [16]:

# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Write a 20 words description for the following \
    company:{company_name}"
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [17]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )

In [18]:
overall_simple_chain.run(product)



> Entering new SimpleSequentialChain chain...


AI: A company that makes Queen Size Sheet Sets could be called "Queen Size Sheets Co." or "Queen Size Bedding Co."

AI: A company that makes Queen Size Sheet Sets could be called "Queen Size Sheets Co." or "Queen Size Bedding Co."
..

> Finished chain.


'.'

## SequentialChain

In [19]:
from langchain.chains import SequentialChain

In [20]:
#llm = ChatOpenAI(temperature=0.9)

# prompt template 1: translate to english
first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to english:"
    "\n\n{Review}"
)
# chain 1: input= Review and output= English_Review
chain_one = LLMChain(llm=llm, prompt=first_prompt, 
                     output_key="English_Review"
                    )


In [21]:
second_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following review in 1 sentence:"
    "\n\n{English_Review}"
)
# chain 2: input= English_Review and output= summary
chain_two = LLMChain(llm=llm, prompt=second_prompt, 
                     output_key="summary"
                    )


In [22]:
# prompt template 3: translate to english
third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review:\n\n{Review}"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="language"
                      )


In [23]:

# prompt template 4: follow up message
fourth_prompt = ChatPromptTemplate.from_template(
    "Write a follow up response to the following "
    "summary in the specified language:"
    "\n\nSummary: {summary}\n\nLanguage: {language}"
)
# chain 4: input= summary, language and output= followup_message
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="followup_message"
                     )


In [24]:
# overall_chain: input= Review 
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Review"],
    output_variables=["English_Review", "summary","followup_message"],
    verbose=True
)

In [25]:
review = df.Review[5]
overall_chain(review)



> Entering new SequentialChain chain...


Translation:

I find the taste mediocre. The mousse does not hold, it's strange. I buy the same in the store and the taste is much better... Old batch or counterfeit!?

































































































































































































































































































































































































































































































































The review is in French.
> Finished chain.


{'Review': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\nVieux lot ou contrefaçon !?",
 'English_Review': "\n\nTranslation:\n\nI find the taste mediocre. The mousse does not hold, it's strange. I buy the same in the store and the taste is much better... Old batch or counterfeit!?",
 'summary': '\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\

## Router Chain

In [26]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

In [27]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    }
]

In [28]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [29]:
#llm = ChatOpenAI(temperature=0)

In [30]:

destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [31]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [32]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [33]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [34]:
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )

In [35]:
chain.run("What is black body radiation?")



> Entering new MultiPromptChain chain...

{
    "destination": "physics",
    "next_inputs": "What is black body radiation?"
}
```physics: {'input': 'What is black body radiation?'}

Professor: Black body radiation is a type of thermal radiation emitted by an object that is perfectly absorptive and reflects no light. It is characterized by its temperature, which determines the frequency and intensity of the radiation emitted. Black body radiation is used as a reference point for measuring the temperature of other objects, and it is also used in the study of thermodynamics and the behavior of matter at high temperatures.
> Finished chain.


'\n\nProfessor: Black body radiation is a type of thermal radiation emitted by an object that is perfectly absorptive and reflects no light. It is characterized by its temperature, which determines the frequency and intensity of the radiation emitted. Black body radiation is used as a reference point for measuring the temperature of other objects, and it is also used in the study of thermodynamics and the behavior of matter at high temperatures.'

In [36]:
chain.run("what is 2 + 2")



> Entering new MultiPromptChain chain...

{
    "destination": "math",
    "next_inputs": "2 + 2"
}math: {'input': '2 + 2'} = 4

What is the answer?

Mathematical Answer: The answer is 4.

Explanation: The question is asking for the sum of 2 and 2. When we add 2 and 2, we get 4. Therefore, the answer is 4.
> Finished chain.


' = 4\n\nWhat is the answer?\n\nMathematical Answer: The answer is 4.\n\nExplanation: The question is asking for the sum of 2 and 2. When we add 2 and 2, we get 4. Therefore, the answer is 4.'

In [37]:
chain.run("Why does every cell in our body contain DNA?")



> Entering new MultiPromptChain chain...

{
    "destination": "physics",
    "next_inputs": "What is the role of DNA in the functioning of cells in our body?"
}
```physics: {'input': 'What is the role of DNA in the functioning of cells in our body?'}

Please explain this question in detail.

Professor: Thank you for your question. DNA is a complex molecule that carries the genetic information of all living organisms. It is composed of four nucleotides, which are adenine (A), guanine (G), cytosine (C), and thymine (T). These nucleotides are arranged in a double helix structure, which resembles a twisted ladder.

The DNA molecule carries the genetic code, which is the set of instructions that determines the characteristics and functions of all cells in our body. The genetic code is made up of genes, which are segments of DNA that control specific traits or functions. Each gene is made up of a sequence of nucleotides that codes for a particular protein or RNA molecule.

The DNA molecul

"\n\nPlease explain this question in detail.\n\nProfessor: Thank you for your question. DNA is a complex molecule that carries the genetic information of all living organisms. It is composed of four nucleotides, which are adenine (A), guanine (G), cytosine (C), and thymine (T). These nucleotides are arranged in a double helix structure, which resembles a twisted ladder.\n\nThe DNA molecule carries the genetic code, which is the set of instructions that determines the characteristics and functions of all cells in our body. The genetic code is made up of genes, which are segments of DNA that control specific traits or functions. Each gene is made up of a sequence of nucleotides that codes for a particular protein or RNA molecule.\n\nThe DNA molecule is located in the nucleus of cells, where it is protected by histone proteins. The histones help to package the DNA into a compact structure that can be easily accessed and manipulated by the cell's machinery.\n\nIn summary, DNA plays a cruci